In [ ]:
class MixedHyperModel(HyperModel):
    def __init__(self, n_timesteps, n_features):
        self.n_timesteps = n_timesteps
        self.n_features = n_features

    def build(self, hp):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(hp.Int('lstm_units_1', 50, 500, step=50), return_sequences=True,
                       input_shape=(self.n_timesteps, self.n_features), kernel_initializer='glorot_uniform'), unroll=False)

        # Aggiunta di layer LSTM intermedi
        for i in range(hp.Int('num_lstm_layers', 1, 4)):
            model.add(LSTM(hp.Int(f'lstm_units_{i+2}', 50, 500, step=50), return_sequences=True,
                           kernel_regularizer=l2(hp.Float(f'l2_rate_{i+2}', 0.000001, 5, step=0.001)), kernel_initializer='glorot_uniform'), unroll=False)
            model.add(Dropout(hp.Float(f'lstm_dropout_{i+2}', 0, 0.5, step=0.1)))

        # Ultimo layer LSTM prima del layer Dense
        model.add(LSTM(hp.Int(f'lstm_units_{i+2}', 50, 500, step=50), return_sequences=False,
                                       kernel_regularizer=l2(hp.Float(f'l2_rate_{i+2}', 0.000001, 5, step=0.001)), kernel_initializer='glorot_uniform'), unroll=False)

        # Aggiunta di layer Dense
        for i in range(hp.Int('num_dense_layers', 1, 4)):
            model.add(Dense(hp.Int(f'dense_units_{i}', 50, 500, step=50), activation='relu', kernel_initializer='glorot_uniform'))
            model.add(Dropout(hp.Float(f'dense_dropout_{i}', 0, 0.5, step=0.1)))

        model.add(Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform'))

        model.compile(optimizer="adam", 
              loss='binary_crossentropy', 
              metrics=['accuracy', 
                       Precision(name='precision'), 
                       Recall(name='recall'), 
                       AUC(name='auc'),
                       tf.keras.metrics.F1Score(threshold=0.5)])

        return model

def bay(hypermodel, objective, X_train, Y_train, X_val, Y_val, epochs, batch_size, callbacks):
    bayesian_tuner = BayesianOptimization(
        hypermodel,
        objective=objective,
        max_trials=50,
        directory='bayesian_search',
        project_name='bayesian_tuning'
    )
    
    bayesian_tuner.search_space_summary()    

    bayesian_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=callbacks)
    
    best_model_bayesian = bayesian_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_bayesian = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Bayesian Optimization:", best_hyperparameters_bayesian.values)
    
    return bayesian_tuner, best_model_bayesian, best_hyperparameters_bayesian

def hb(hypermodel, X_train, Y_train, X_val, Y_val):
    hyperband_tuner = kt.tuners.Hyperband(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        directory='hyperband_search',
        project_name='hyperband_tuning'
    )
    
    hyperband_tuner.search_space_summary()
    
    hyperband_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_hyperband = hyperband_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_hyperband = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Hyperband:", best_hyperparameters_hyperband.values)
    
    return best_model_hyperband, best_hyperparameters_hyperband

def rd(hypermodel, X_train, Y_train, X_val, Y_val):
    random_tuner = kt.tuners.RandomSearch(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        directory='random_search',
        project_name='random_tuning'
    )
    random_tuner.search_space_summary()
    
    random_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    # Ottieni il miglior modello e i migliori iperparametri
    best_model_random = random_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_random = random_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Random Search:", best_hyperparameters_random.values)
    
    return best_model_random, best_hyperparameters_random